In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
PATH = '/content/gdrive/MyDrive/Risk_Aware_RL'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
#@title Initialization
import os
import time
import shutil
from tqdm import tqdm
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

def printf(text, file_path):
  with open(file_path, 'a') as file:
      # Print to console
      print(text)
      # Write to the file
      file.write(text + '\n')

tests = 1 #@param {allow-input:true}
tests = int(tests)

file_start_num = 1 #@param {allow-input:true}
file_start_num = int(file_start_num)


# Experiment Parameters
safe_padding = True
max_iterations = 400

# Hyper Parameters
discount_factor_Q = 0.9
alpha = 0.85
observation_radius = 2
temp = 0.005

choice_of_C = "ThesisDecreasing"

# Choice of Prior
prior_choice = "Uninformative Prior" #@param ["Uninformative Prior", "Medium Informative Prior", "High Informative Prior"] {allow-input:false}
pc = None
if prior_choice == "Uninformative Prior":
  # critical_threshold = 0.33
  critical_threshold = 0.01
  pc = "1"
elif prior_choice == "Medium Informative Prior":
  critical_threshold = 0.33
  # critical_threshold = 0.01
  pc = "2"
elif prior_choice == "High Informative Prior":
  critical_threshold = 0.0033
  pc = "3"
else:
  raise Exception("Check your prior choice configuration")

now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")

folder_name = f"{tests}runs_({file_start_num}_to_{str(file_start_num+tests-1)})_bridge_Pmax{critical_threshold}_C{choice_of_C}_Prior{prior_choice[0:4]}_experiment_"

basepath = os.path.join(PATH, 'runs',f"prior_{pc}", folder_name+dt_string)
results_path = os.path.join(basepath, 'Results')
log = os.path.join(basepath, "log.txt")
if not os.path.isdir(basepath):
    os.makedirs(basepath)
if not os.path.isdir(results_path):
    os.makedirs(results_path)
printf(f"Folder: {basepath}", log)



if safe_padding:
  # number_of_runs = 500
  number_of_runs = 1500
  printf(f"\n#############################\n"
        f"safe_padding: {safe_padding}\n"
        f"number_of_runs: {number_of_runs}\n"
        f"max_iterations: {max_iterations}\n"
        f"discount_factor_Q: {discount_factor_Q}\n"
        f"alpha: {alpha}\n"
        f"observation_radius: {observation_radius}\n"
        f"temp: {temp}\n"
        f"prior_choice: {prior_choice}\n"
        f"critical_threshold: {critical_threshold}\n"
        f"choice_of_C: {choice_of_C}\n"
        f"#############################\n", log)
else:
  number_of_runs = 1500
  printf(f"\n#############################\n"
        f"safe_padding: {safe_padding}\n"
        f"number_of_runs: {number_of_runs}\n"
        f"max_iterations: {max_iterations}\n"
        f"discount_factor_Q: {discount_factor_Q}\n"
        f"alpha: {alpha}\n"
        f"observation_radius: {observation_radius}\n"
        f"temp: {temp}\n"
        f"prior_choice: {prior_choice}\n"
        f"critical_threshold: {critical_threshold}\n"
        f"choice_of_C: {choice_of_C}\n"
        f"Normal_Q_Learning_OnlyVersion!!!!!!!\n"
        f"#############################\n", log)
save_runs = 100
printf(f"You want {tests} tests.", log)
printf(f"Breakpoint will be saved every {save_runs} steps", log)

Folder: /content/gdrive/MyDrive/Risk_Aware_RL/runs/prior_1/1runs_(1_to_1)_bridge_Pmax0.01_CThesisDecreasing_PriorUnin_experiment_20230809_201239

#############################
safe_padding: True
number_of_runs: 1500
max_iterations: 400
discount_factor_Q: 0.9
alpha: 0.85
observation_radius: 2
temp: 0.005
prior_choice: Uninformative Prior
critical_threshold: 0.01
choice_of_C: ThesisDecreasing
#############################

You want 1 tests.
Breakpoint will be saved every 100 steps


In [ ]:
#@title Functions

# save_ragged
def stack_ragged(array_list, axis=0):
    lengths = [np.shape(a)[axis] for a in array_list]
    idx = np.cumsum(lengths[:-1])
    stacked = np.concatenate(array_list, axis=axis)
    return stacked, idx

def save_stacked_array(fname, array_list, axis=0):
    stacked, idx = stack_ragged(array_list, axis=axis)
    np.savez(fname, stacked_array=stacked, stacked_index=idx)

def load_stacked_arrays(fname, axis=0):
    npzfile = np.load(fname)
    idx = npzfile['stacked_index']
    stacked = npzfile['stacked_array']
    return np.split(stacked, idx, axis=axis)

def automaton(u_val, last_automaton_state):
    # 1 is unsafe and 2 is target
    # If we were on a neutral state
    if last_automaton_state == 0:
        next_automaton_state = u_val
    else:
        # We were already either unsafe or target
        next_automaton_state = last_automaton_state
    return next_automaton_state


def take_action_m(current_location, action_indx, is_det, u):
    # action is initialized at (19, 0)
    next_location = current_location.copy()
    if not is_det:
        if np.random.uniform() > .95:
            action_indx = np.floor(5 * np.random.uniform())
    if action_indx == 0:  # Right
        next_location[1] = current_location[1] + 1
    elif action_indx == 1:  # Up
        next_location[0] = current_location[0] - 1
    elif action_indx == 2:  # Left
        next_location[1] = current_location[1] - 1
    elif action_indx == 3:  # Down
        next_location[0] = current_location[0] + 1
    elif action_indx == 4:  # Top Right
        next_location[0] -= 1
        next_location[1] += 1
    elif action_indx == 5:  # Top Left
        next_location[0] -= 1
        next_location[1] -= 1
    elif action_indx == 6:  # Lower Left
        next_location[0] += 1
        next_location[1] -= 1
    elif action_indx == 7:  # Lower Right
        next_location[0] += 1
        next_location[1] += 1
    # Else if action is 8, Stay.

    # map locations within the workspace
    if next_location[0] > 19:
        next_location[0] = 19
    if next_location[0] < 0:
        next_location[0] = 0
    if next_location[1] > 19:
        next_location[1] = 19
    if next_location[1] < 0:
        next_location[1] = 0
    next_location[2] = automaton(u[next_location[0], next_location[1]], current_location[2])
    return next_location


def take_action_m_boundary(current_location, action_indx, is_det, u):
    out_of_bounds = False
    next_location = current_location.copy()
    if (not is_det):
        if (np.random.uniform() > .95):
            action_indx = np.floor(5 * np.random.uniform())
    if action_indx == 0:  # Right
        next_location[1] = current_location[1] + 1
    elif action_indx == 1:  # Up
        next_location[0] = current_location[0] - 1
    elif action_indx == 2:  # Left
        next_location[1] = current_location[1] - 1
    elif action_indx == 3:  # Down
        next_location[0] = current_location[0] + 1
    elif action_indx == 4:  # Top Right
        next_location[0] -= 1
        next_location[1] += 1
    elif action_indx == 5:  # Top Left
        next_location[0] -= 1
        next_location[1] -= 1
    elif action_indx == 6:  # Lower Left
        next_location[0] += 1
        next_location[1] -= 1
    elif action_indx == 7:  # Lower Right
        next_location[0] += 1
        next_location[1] += 1
    # Else if action is 8, Stay.

    if next_location[0] > 19:
        next_location[0] = 19
        out_of_bounds = True
    if next_location[0] < 0:
        next_location[0] = 0
        out_of_bounds = True
    if next_location[1] > 19:
        next_location[1] = 19
        out_of_bounds = True
    if next_location[1] < 0:
        next_location[1] = 0
        out_of_bounds = True
    next_location[2] = automaton(u[next_location[0], next_location[1]], current_location[2])
    return next_location, out_of_bounds


# For vectorizing local_U_update_fixed_actions
def take_actions_boundary_det(current_location, u):
    out_of_bounds = np.zeros(9)
    next_locations = np.tile(current_location, (9, 1))
    action_to_movement = np.array([[0, 1], [-1, 0], [0, -1], [1, 0], [-1, 1], [-1, -1], [1, -1], [1, 1], [0, 0]])
    next_locations[:, 0:2] += action_to_movement[[0, 1, 2, 3, 4, 5, 6, 7, 8]]
    # action_to_movement = np.array([[0, 1], [-1, 0], [0, -1], [1, 0], [0, 0]])
    # next_locations[:, 0:2] += action_to_movement[[0, 1, 2, 3, 4]]

    # Couldnt figure out a good way around this loop
    for i in np.arange(9):
        if next_locations[i, 0] > 19:
            next_locations[i, 0] = 19
            out_of_bounds[i] = True
        if next_locations[i, 0] < 0:
            next_locations[i, 0] = 0
            out_of_bounds[i] = True
        if next_locations[i, 1] > 19:
            next_locations[i, 1] = 19
            out_of_bounds[i] = True
        if next_locations[i, 1] < 0:
            next_locations[i, 1] = 0
            out_of_bounds[i] = True
        next_locations[i, 2] = automaton(u[next_locations[i, 0], next_locations[i, 1]], current_location[2])
    return next_locations, out_of_bounds


def take_action_m_direction(current_location, action_indx, is_det, u):
    next_location = current_location.copy()
    if (not is_det):
        if (np.random.uniform() > .95):
            action_indx = np.int(5 * np.random.uniform())
    if action_indx == 0:  # Right
        next_location[1] = current_location[1] + 1
    elif action_indx == 1:  # Up
        next_location[0] = current_location[0] - 1
    elif action_indx == 2:  # Left
        next_location[1] = current_location[1] - 1
    elif action_indx == 3:  # Down
        next_location[0] = current_location[0] + 1
    elif action_indx == 4:  # Top Right
        next_location[0] -= 1
        next_location[1] += 1
    elif action_indx == 5:  # Top Left
        next_location[0] -= 1
        next_location[1] -= 1
    elif action_indx == 6:  # Lower Left
        next_location[0] += 1
        next_location[1] -= 1
    elif action_indx == 7:  # Lower Right
        next_location[0] += 1
        next_location[1] += 1
    # Else if action is 8, Stay.

    direction_taken = action_indx
    if next_location[0] > 19:
        next_location[0] = 19
        direction_taken = 8
    if next_location[0] < 0:
        next_location[0] = 0
        direction_taken = 8
    if next_location[1] > 19:
        next_location[1] = 19
        direction_taken = 8
    if next_location[1] < 0:
        next_location[1] = 0
        direction_taken = 8
    next_location[2] = automaton(u[next_location[0], next_location[1]], current_location[2])
    return next_location, direction_taken


# Reward function
def Q_r(next_state):
    if next_state[2] == 2:
        return 1
    else:
        return 0


# Local risk calcuation with fixed actions
def local_U_update_fixed_actions(current_state, neighbours, unsafe_states, depth, ps_delta, min_actions,
                                 out_of_bounds_vec, next_state_indx_vec, in_neighbours_vec, current_out_of_bounds,
                                 current_next_states_indx_vec, current_in_neighbours_vec, U_start):
    discount = 1
    U = U_start.copy()

    # First depth-1 steps at once, with actions given by min_actions
    for d in np.arange(depth - 1):
        actions = min_actions[d, :]
        sums = discount * ps_delta[neighbours[:, 0], neighbours[:, 1], actions, :] * U[next_state_indx_vec[:, :]] * (
                out_of_bounds_vec[:, :] == 0) * in_neighbours_vec[:, :]
        U = np.sum(sums, axis=1)

        # Ensure U is 1 for unsafe states.
        U[unsafe_states] = 1

    # FINAL STEP (fixed all actions)
    sums1 = discount * ps_delta[current_state[0], current_state[1], :, :] * U[current_next_states_indx_vec[:]] * (
            current_out_of_bounds == 0) * current_in_neighbours_vec

    U_delta = np.sum(sums1, 1)
    return U_delta


# Variance on Expected Risk Calculation
def risk_variance(current_state, neighbours, unsafe_states, num_neighbours, depth, ps, cov, min_actions, U_s_a,
                  out_of_bounds_vec, next_state_indx_vec, in_neighbours_vec, current_out_of_bounds,
                  current_next_states_indx_vec, current_in_neighbours_vec, U_start):
    # Covariance Matrix based on lexicographic ordering of state,action,state on
    # transition probabilities
    # We only represent the non-zero entries in the covariance matrix.
    # So cov(q1,q2,a,direction1,direction2) gives the covariance between the
    # transitions probability (q1,q2) (a)-> direction1(q1,q2) and (q1,q2) (a)-> direction2(q1,q2)
    # Maybe we should maintain and update this rather than recalculating at
    # every step.

    # We choose delta for numerical differentiation, standard detla = 0.001
    delta = 0.001

    # Just calculate the relevant ones (transition probs starting in
    # neighbouring states - these are the only ones used in risk
    variance_U = np.zeros(9)

    # Now calculate variance
    # Do simultaneously for each first_action
    for neigh in np.arange(num_neighbours):
        for a in np.arange(9):
            # For storing gradients
            grad = np.zeros((9, 9))
            for d in np.arange(9):
                # Calculate derivative wrt. p_neigh_a_d
                ps_delta = ps.copy()
                ps_delta[neighbours[neigh, 0], neighbours[neigh, 1], a, d] = \
                    ps_delta[neighbours[neigh, 0], neighbours[neigh, 1], a, d] + delta

                # Replicate local_U_update calculation but with actions
                # according to min_actions.

                U_delta = local_U_update_fixed_actions(current_state, neighbours, unsafe_states, depth, ps_delta,
                                                       min_actions, out_of_bounds_vec, next_state_indx_vec,
                                                       in_neighbours_vec, current_out_of_bounds,
                                                       current_next_states_indx_vec, current_in_neighbours_vec, U_start)
                # Row? vector of gradients for risk after each first_action, wrt
                # p_neigh_a_d.
                # Each column contains the gradients wrt p_neigh_a_allds for the
                # risk after a particular action
                grad[d, :] = (U_delta - U_s_a) / delta

            # Add contribution to variance. @ is matrix multiplication
            for first_action in np.arange(9):
                variance_U[first_action] = variance_U[first_action] + np.transpose(grad[:, first_action]) @ cov[
                                                                                                            neighbours[
                                                                                                                neigh, 0],
                                                                                                            neighbours[
                                                                                                                neigh, 1],
                                                                                                            a, :,
                                                                                                            :] @ grad[:,
                                                                                                                 first_action]
    return variance_U


# Local Risk Calculation
def local_U_update(current_state, neighbours, depth, u, ps, cov):
    discount = 1
    num_neighbours = np.shape(neighbours)[0]

    # Pre-Calculate as Much as We Can.
    U_start = np.zeros(num_neighbours)
    unsafe_states = neighbours[:, 2] == 1
    U_start[unsafe_states] = 1

    # Pre-calculate next_state, next_state_indx, out_of_bounds and in_neighbours
    # for each neigh and direction
    next_state_vec = np.zeros((num_neighbours, 9, 3), dtype=np.int8)
    out_of_bounds_vec = np.zeros((num_neighbours, 9), np.int8)
    next_state_indx_vec = np.zeros((num_neighbours, 9),
                                   np.int8)  # Defaults to 0 if not in neighbours. Won't influence calculation
    in_neighbours_vec = np.ones((num_neighbours, 9), np.int8)
    for neigh in np.arange(num_neighbours):
        for direction in np.arange(9):
            next_state, out_of_bounds = take_action_m_boundary(neighbours[neigh], direction, True, u)
            next_state_vec[neigh, direction] = next_state
            out_of_bounds_vec[neigh, direction] = out_of_bounds

            (next_state_indx_array,) = np.where(np.all(neighbours == next_state, axis=1))

            if len(next_state_indx_array) == 0:
                in_neighbours_vec[neigh, direction] = 0
                # next_state_indx_vec defaults to 0
            else:
                # in_neighbours defaults to 1
                next_state_indx_vec[neigh, direction] = next_state_indx_array[0]

    # Also pre-calculate next_states, next_states_indx, out_of_bounds and in_neighbours
    # For the final step, with CURRENT STATE and All Directions
    current_next_states, current_out_of_bounds = take_actions_boundary_det(current_state,
                                                                           u)  # Note that out_of_bounds here is 1 dimensional, but is used in the multiplication as 2 dimensional, because it is the same over actions.
    current_next_states_indx_vec = np.zeros(9, dtype=np.int8)  # Same comment
    current_in_neighbours_vec = np.ones(9, dtype=np.int8)  # Same comment
    # Now I want to get the list of the indices of current_next_states in neighbours.
    for direction in np.arange(9):
        (current_next_state_indx_array,) = np.where(np.all(neighbours == current_next_states[direction, :], axis=1))
        if len(current_next_state_indx_array) == 0:
            current_in_neighbours_vec[direction] = 0
        else:
            current_next_states_indx_vec[direction] = current_next_state_indx_array[0]

    U_temp = U_start.copy()
    U = U_start.copy()

    # First depth-1 steps (minimum expected risk action)
    # Saves the minimum expected risk action for each state.
    min_actions = np.zeros((depth - 1, num_neighbours), dtype=np.int8)
    for d in np.arange(depth - 1):
        for neigh in np.arange(num_neighbours):
            sums = np.zeros((9, 9))
            for direction in np.arange(9):
                next_state = next_state_vec[neigh, direction]
                out_of_bounds = out_of_bounds_vec[neigh, direction]
                next_state_indx = next_state_indx_vec[neigh, direction]
                in_neighbours = in_neighbours_vec[neigh, direction]

                sums[:, direction] = discount * ps[neighbours[neigh, 0], neighbours[neigh, 1], :, direction] * U[
                    next_state_indx] * (out_of_bounds == 0) * in_neighbours

            action_risks = np.sum(sums, 1)
            chosen_action = np.argmin(action_risks)
            U_temp[neigh] = action_risks[chosen_action]
            min_actions[d, neigh] = chosen_action

        U = U_temp.copy()
        # Ensure U is 1 for unsafe states.
        U[unsafe_states] = 1
        U_temp[unsafe_states] = 1

    # Final Step (fixed action) (just current state)
    sums1 = discount * ps[current_state[0], current_state[1], :, :] * U[current_next_states_indx_vec[:]] * (
            current_out_of_bounds == 0) * current_in_neighbours_vec
    U_s_a = np.sum(sums1, 1)

    variance_U = risk_variance(current_state, neighbours, unsafe_states, num_neighbours, depth, ps, cov, min_actions,
                               U_s_a, out_of_bounds_vec, next_state_indx_vec, in_neighbours_vec, current_out_of_bounds,
                               current_next_states_indx_vec, current_in_neighbours_vec, U_start)
    return U_s_a, variance_U


In [ ]:
#@title Training
cnt = 1
for experiment_number in range(file_start_num, file_start_num+int(tests)):
    experiment_name = "bridge_Pmax{Pmax}_C{C}_Prior{prior}_experiment{number}".format(
        Pmax=critical_threshold, prior=prior_choice[0:4], C=choice_of_C, number=experiment_number)
    printf(f"\n-------- Experiment ({cnt} of {tests}) ---------\nName: {experiment_name}", log)
    cnt += 1

    # 20 by 20 grid
    X_movable_limit = 20
    Y_movable_limit = 20
    X = np.linspace(1, X_movable_limit, X_movable_limit)
    Y = np.linspace(1, Y_movable_limit, Y_movable_limit)
    [gridx, gridy] = np.meshgrid(X, Y)

    # State Labels
    u = np.zeros([20, 20])
    u[0:7, 0:20] = 2  # goal zone
    u[8:12, 0:8] = 1  # dangerous area
    u[8:12, 11:20] = 1  # dangerous area

    # Initialize Counters/Trackers
    it_run_num = 1
    number_of_fails = 0
    failed_path = []
    fail_history = []
    success_history = []
    number_of_successes = 0
    path_history = []
    successful_runs = np.zeros(number_of_runs)

    # Q function. A 4-dim matrix (action, state(X, Y), automaton_state)
    total_number_of_actions = 9
    Q = np.zeros((total_number_of_actions, X_movable_limit, Y_movable_limit, 3))
    # Is there a point having the automaton_state here? Maybe just for generality?

    # State Counts
    state_count = np.zeros((X_movable_limit, Y_movable_limit))
    state_count[19, 0] = 1  # initial state

    # posterior-prior = counts
    # Defining the Prior
    # Agent's belief about transition probability
    if prior_choice == "Uninformative Prior":
        state_action_direction_prior = np.zeros((X_movable_limit, Y_movable_limit,
                                                 total_number_of_actions, total_number_of_actions))
        for action_indx in np.arange(total_number_of_actions):
            for x in np.arange(0, X_movable_limit):
                for y in np.arange(0, Y_movable_limit):
                    for direction in np.arange(0, total_number_of_actions):
                        poss_next_state, out_of_bounds = take_action_m_boundary([x, y, 1], direction, True, u)
                        if out_of_bounds == False:
                            state_action_direction_prior[x, y, action_indx, direction] = 1
        state_action_direction_posterior = state_action_direction_prior

    if prior_choice == "Medium Informative Prior":
        state_action_direction_prior = np.zeros((X_movable_limit, Y_movable_limit,
                                                 total_number_of_actions, total_number_of_actions))
        for action_indx in np.arange(total_number_of_actions):
            for x in np.arange(0, X_movable_limit):
                for y in np.arange(0, Y_movable_limit):
                    for direction in np.arange(0, total_number_of_actions):
                        poss_next_state, out_of_bounds = take_action_m_boundary([x, y, 1], direction, True, u)
                        if out_of_bounds == False:
                            if action_indx == direction:
                                state_action_direction_prior[x, y, action_indx, direction] = 12
                            else:
                                state_action_direction_prior[x, y, action_indx, direction] = 1
        state_action_direction_posterior = state_action_direction_prior

    if prior_choice == "High Informative Prior":
        state_action_direction_prior = np.zeros((X_movable_limit, Y_movable_limit,
                                                 total_number_of_actions, total_number_of_actions))
        for action_indx in np.arange(total_number_of_actions):
            for x in np.arange(0, X_movable_limit):
                for y in np.arange(0, Y_movable_limit):
                    for direction in np.arange(0, total_number_of_actions):
                        poss_next_state, out_of_bounds = take_action_m_boundary([x, y, 1], direction, True, u)
                        if out_of_bounds == False:
                            if action_indx == direction:
                                state_action_direction_prior[x, y, action_indx, direction] = 96
                            else:
                                state_action_direction_prior[x, y, action_indx, direction] = 1
        state_action_direction_posterior = state_action_direction_prior

    # State_action_prior: similar to the above, based on the prior given above
    state_action_prior = np.sum(state_action_direction_prior, 3)
    state_action_posterior = state_action_prior  # posterior is initialized the same as prior

    # Expected transition probabilities
    ps = np.divide(state_action_direction_prior, np.expand_dims(state_action_prior, 3))

    # Covariance Matrix: cov(q1,q2,action,:,:) is the 5*5 covariance matrix for
    # the dirichlet for taking action at state (q1,q2)
    cov = np.zeros(
        (X_movable_limit, Y_movable_limit, total_number_of_actions, total_number_of_actions, total_number_of_actions))
    for q1 in np.arange(0, X_movable_limit):
        for q2 in np.arange(0, Y_movable_limit):
            for a in np.arange(0, total_number_of_actions):
                for direction_i in np.arange(0, total_number_of_actions):
                    for direction_j in np.arange(0, total_number_of_actions):
                        cov[q1, q2, a, direction_i, direction_j] = ((direction_i == direction_j) * ps[
                            q1, q2, a, direction_i] - ps[q1, q2, a, direction_i] * ps[q1, q2, a, direction_j]) \
                                                                   / (state_action_posterior[q1, q2, a] + 1)

    # Defining the parameter C as a function of state_count
    if choice_of_C == "ThesisDecreasing":
        def C_function(cur_state_count):
            return 0.7 * max((24 - cur_state_count) / 25, 0) + 0.3 / (1 + cur_state_count)

    if choice_of_C[0:5] == "Fixed":
        c = float(choice_of_C[5:9])

        def C_function(cur_state_cout):
            return c

    if choice_of_C[0:5] == "Slope":
        num = int(choice_of_C[5:7])

        def C_function(cur_state_count):
            return 0.7 * max(((num - 1) - cur_state_count) / num, 0) + 0.3 / (1 + cur_state_count)

    # Train Agent
    start_time = time.time()
    for run_num in tqdm(np.arange(1, number_of_runs + 1)):
        current_state = np.array([19, 0, 0])  # In the product MDP
        current_path = np.expand_dims(current_state, 0)  # Add a dimension so we can keep a list of states
        # printf('Run Number: ' + str(run_num), log)
        print(f"\nRun Number -> {run_num}")
        it_num = 1
        while (current_state[-1] != 2) and (it_num <= max_iterations):
            it_num = it_num + 1
            it_run_num = it_run_num + 1
            """
            product MDP: (x, y) as well as the automaton state (where automaton means if the robot is unsafe now)
            """

            if safe_padding:
                depth = 2  # Must be <= observation radius
                old_neighbours = np.expand_dims(current_state, 0)  # Add a dimension so we can keep a list of states
                neighbours = np.array([]).astype(np.int8).reshape(0, 3)
                # CAREFUL IF THE SIZE OF THE AREA IS LARGER THAN 128
                # Expand neighbours one at a time until covering the
                # Observation_radius: make sure it is unique.
                # NOTE: Neighbours is in the product MDP
                for i in np.arange(observation_radius):
                    for current_exploring in np.arange(np.size(old_neighbours, 0)):
                        for action in np.arange(total_number_of_actions):
                            neighbours = np.vstack(
                                [neighbours, take_action_m(old_neighbours[current_exploring], action, True, u)])
                        neighbours = np.unique(neighbours, axis=0)
                    old_neighbours = np.vstack([old_neighbours, neighbours])
                    old_neighbours = np.unique(old_neighbours, axis=0)

                # RISK AND VARIANCE CALCULATION!
                U, variance_U = local_U_update(current_state, neighbours, depth, u, ps, cov)

                # Defining the parameter C for risk-averseness based on state_count
                C = C_function(state_count[current_state[0], current_state[1]])

                # Perform Cantelli Bound calculation
                P_a = U + np.sqrt((variance_U * C) / (1 - C))
                (acceptable_actions,) = np.nonzero(P_a < critical_threshold)
                if acceptable_actions.size == 0:
                    # If nothing is acceptable, then just choose minimum U.
                    acceptable_actions = np.nonzero(U == np.amin(U))

            available_Qs = np.zeros(total_number_of_actions)
            if safe_padding:
                for each_action in acceptable_actions:
                    available_Qs[each_action] = Q[each_action, current_state[0], current_state[1], current_state[2]]
                for each_action in np.setdiff1d(np.arange(total_number_of_actions), acceptable_actions, assume_unique=True):
                    # Find the set difference of two arrays
                    available_Qs[each_action] = Q[each_action, current_state[0], current_state[1], current_state[2]] - 300
            else:
                for each_action in np.arange(total_number_of_actions):
                    available_Qs[each_action] = Q[each_action, current_state[0], current_state[1], current_state[2]]

            # Boltzmann rational
            expo = np.exp(available_Qs / temp)
            probabs = expo / sum(expo)

            # Select an action
            actions = np.arange(total_number_of_actions)
            selected_action = np.random.choice(actions, p=probabs)

            # Take that action
            next_state, direction_taken = take_action_m_direction(current_state, selected_action, False, u)

            # UPDATES
            # Update posterior by adding counts
            state_action_direction_posterior[current_state[0], current_state[1], selected_action, direction_taken] = \
                state_action_direction_posterior[current_state[0], current_state[1], selected_action, direction_taken] + 1

            state_action_posterior[current_state[0], current_state[1], selected_action] = \
                state_action_posterior[current_state[0], current_state[1], selected_action] + 1
            # Update Means
            ps[current_state[0], current_state[1], selected_action, :] = \
                state_action_direction_posterior[current_state[0], current_state[1], selected_action, :] / \
                state_action_posterior[current_state[0], current_state[1], selected_action]
            # Update Covariance Matrix
            for direction_i in np.arange(total_number_of_actions):
                for direction_j in np.arange(total_number_of_actions):
                    cov[current_state[0], current_state[1], selected_action, direction_i, direction_j] = ((
                                                                                                                  direction_i == direction_j) *
                                                                                                          ps[current_state[
                                                                                                                 0],
                                                                                                             current_state[
                                                                                                                 1], selected_action, direction_i] -
                                                                                                          ps[current_state[
                                                                                                                 0],
                                                                                                             current_state[
                                                                                                                 1], selected_action, direction_i] *
                                                                                                          ps[current_state[
                                                                                                                 0],
                                                                                                             current_state[
                                                                                                                 1], selected_action, direction_j]) \
                                                                                                         / (
                                                                                                                 state_action_posterior[
                                                                                                                     current_state[
                                                                                                                         0],
                                                                                                                     current_state[
                                                                                                                         1], selected_action] + 1)

            # Update Q function
            current_Qs = Q[:, next_state[0], next_state[1], next_state[2]]

            Q[selected_action, current_state[0], current_state[1], current_state[2]] = (1 - alpha) * \
                                                                                       Q[selected_action, current_state[0],
                                                                                         current_state[1], current_state[
                                                                                             2]] + alpha * \
                                                                                       (
                                                                                               Q_r(next_state) + discount_factor_Q * np.amax(
                                                                                           current_Qs))

            # Update state counts to reflect moving to next state
            state_count[next_state[0], next_state[1]] = state_count[next_state[0], next_state[1]] + 1
            current_path = np.vstack([current_path, next_state])

            # DISPLAY (AND TERMINATE IF FAILED)
            if next_state[2] == 1:
                number_of_fails = number_of_fails + 1
                # print('-------fail-------')
                # print('Current State: ' + str(current_state))
                # print('Next State: ' + str(next_state))
                # print('Neighbours: ' + str(neighbours))
                # print('U: ' + str(U))
                # print('Selected Action:' + str(selected_action))
                break
            elif next_state[2] == 2:
                # print('+++++++success+++++++')
                number_of_successes = number_of_successes + 1
                # Keep a record of which runs were successful.
                successful_runs[run_num - 1] = 1

            # Update current state
            current_state = next_state.copy()

        # Add run to paths.
        path_history.append(current_path)

        # Cumulative total of number of successes.
        fail_history.append(number_of_fails)
        success_history.append(number_of_successes)

        if run_num % int(save_runs) == 0:
          printf(f"\nSaving breakpoint @ experiment number <{experiment_number}> | run <{run_num}>", log)
          np.save(os.path.join(results_path, experiment_name + "_failHistory"), fail_history)
          np.save(os.path.join(results_path, experiment_name + "_successHistory"), success_history)
          np.save(os.path.join(results_path, experiment_name + "_successfulRuns"), successful_runs)
          np.save(os.path.join(results_path, experiment_name + "_Q"), Q)
          save_stacked_array(os.path.join(results_path, experiment_name + "_pathHistory"), path_history, axis=0)

    # Prints
    printf(f"Time Elapsed: {(time.time()-start_time):.2f} secs ({(time.time()-start_time)/60:.2f} mins)", log)
    printf(f"Number of Successes: {str(number_of_successes)}", log)
    printf(f"Number of Failures: {str(number_of_fails)}", log)

    # Save Results!
    np.save(os.path.join(results_path, experiment_name + "_failHistory"), fail_history)
    np.save(os.path.join(results_path, experiment_name + "_successHistory"), success_history)
    np.save(os.path.join(results_path, experiment_name + "_successfulRuns"), successful_runs)
    np.save(os.path.join(results_path, experiment_name + "_Q"), Q)
    save_stacked_array(os.path.join(results_path, experiment_name + "_pathHistory"), path_history, axis=0)
    printf(f"\nTraining Completed.\nEnding experiment number <{experiment_number}>", log)


-------- Experiment (1 of 5) ---------
Name: bridge_Pmax0.33_CThesisDecreasing_PriorMedi_experiment6


  0%|          | 0/500 [00:00<?, ?it/s]


Run Number -> 1


<ipython-input-3-d6bd7564f444>:147: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  action_indx = np.int(5 * np.random.uniform())
  0%|          | 1/500 [00:44<6:10:34, 44.56s/it]


Run Number -> 2


...


Run Number -> 499


100%|█████████▉| 499/500 [39:55<00:03,  3.16s/it]


Run Number -> 500


100%|██████████| 500/500 [39:58<00:00,  4.80s/it]


Saving breakpoint @ experiment number <10> | run <500>
Time Elapsed: 2398.75 secs (39.98 mins)
Number of Successes: 471
Number of Failures: 22

Training Completed.
Ending experiment number <10>
